# MysteryNotebook 3000

> Please comment the following cells for your fellow data scientists - what does each cell do?

> Fill in the missing documentation cells

### Set Credentials and Configure Connection to Spark, Read Input Data.

In [1]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-8f959616-4d92-4073-bbf2-1d3f4f6ffad0',
    'IBM_API_KEY_ID': 'VYkxc-7ohgrp0zdnHRSHAQW-aqI5m-b6j8K3YBllfR4a',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.bluemix.net/oidc/token',
    'BUCKET': 'datascienceinbanking-donotdelete-pr-hrnb5icgks2da6',
    'FILE': 'MysteryData.csv'
}


In [2]:

import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-8f959616-4d92-4073-bbf2-1d3f4f6ffad0',
    'iam_service_endpoint': 'https://iam.bluemix.net/oidc/token',
    'api_key': 'VYkxc-7ohgrp0zdnHRSHAQW-aqI5m-b6j8K3YBllfR4a'
}

configuration_name = 'os_1e498447d5f74cd6b90b92b35bb6514e_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
input_df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('MysteryData.csv', 'datascienceinbanking-donotdelete-pr-hrnb5icgks2da6'))
input_df.show(10, truncate=False)


ModuleNotFoundError: No module named 'ibmos2spark'

## Subset data frame to extract relevant features and associated customer ids

> initialize features and stocks column, to be used later in the pipeline. Subset the data frame to extract categorical variables corresponding to Y/N indicating whether the customer has stock in the industry.

In [4]:
group_index = 'CustID'
item_feature = 'Stocks'
item_features = ['Tech','Auto','Hotel','Airline','Energy','Biotech','Pharma','Utilities','Financials','Staples','Industrials']

if item_features:
    group_items_df = input_df.select([group_index] + item_features)
else:
    group_items_df = input_df.select(group_index, item_feature).distinct()
group_items_df.show(truncate=False)

NameError: name 'input_df' is not defined

## Create a horizontally oriented transaction matrix with transaction id and associated transactions. 

> Initilize true value. Iterate across columns in RDD appending customer id and append values that evaluate to True to an RDD

In [5]:
true_value = 'Y'


if item_features:

    
    def item_list(group):
        item_list = []
        for item in item_features:
            if group[item] == true_value:
                item_list.append(item)
        return item_list

    
    group_items_rdd = group_items_df.rdd.map(lambda group: (group[0], item_list(group)))


else:
    
    
    group_items_rdd = group_items_df.rdd.map(lambda group: (group[0], [group[1]])).reduceByKey(lambda x,y:x+y)

for row in group_items_rdd.take(20):
    print(row)

NameError: name 'group_items_df' is not defined

## Filter items of length<1, to RDD. 

> Filter out customers with stocks only in one industry (no associations). Convert pipeline object to RDD, with customer id as index with one column, 'stocks', where each entry is a list of industries in which the customer holds stock. Display results. 

In [6]:
multi_item_groups_rdd = group_items_rdd.filter(lambda order: len(order[1])>1)

multi_item_groups_df = multi_item_groups_rdd.toDF([group_index, item_feature])
multi_item_groups_df.show(truncate=False)

NameError: name 'group_items_rdd' is not defined

## Perform FPM

> Set Min_support to filter out infrequent item sets. Set the minimum strength of association. Fit model and show frequent item sets attribute of model

In [ ]:
from pyspark.ml.fpm import FPGrowth

min_support=0.05
min_confidence=0.2

fpGrowth = FPGrowth(itemsCol=item_feature, minSupport=min_support, minConfidence=min_confidence)
model = fpGrowth.fit(multi_item_groups_df)
model.freqItemsets.show(truncate=False)

##  Display Generated Association Rules, Confidence Metrics, Lift

In [ ]:
model.associationRules.show(truncate=False)

##  Create a chord graph mapping the associations between antecedents and consequents

In [ ]:
# In Watson Studio
import brunel
pd_associationRules = model.associationRules.toPandas()
%brunel data('pd_associationRules') chord x(antecedent) y(consequent)  color(confidence) size(confidence) tooltip(antecedent, consequent, confidence) :: width=800, height=500

##  Produce network graph

In [ ]:
%%brunel data('pd_associationRules') edge color(confidence:sequential) key(antecedent, consequent) tooltip(antecedent, consequent, confidence) style('symbol:curvedArrow') + network y(antecedent, consequent) key(#values) label(#values) style('.label {font-size:10px}') 
:: width=800, height=600 

##  Call model's transform method on test data to produce a prediction

In [7]:
test_items = ["Auto", "Tech"]

test_data = spark.createDataFrame([(test_items, )], [item_feature])
model.transform(test_data).show(truncate=False)

NameError: name 'spark' is not defined

### References:
- https://spark.apache.org/docs/2.3.0/api/python/pyspark.ml.html?highlight=associationrules#pyspark.ml.fpm.FPGrowth
- https://github.com/apache/spark/blob/master/examples/src/main/python/ml/fpgrowth_example.py


### Developed by Data Science Elite Team, IBM Analytics:

- David Thomason - Software Engineer

Copyright (c) 2018 IBM Corporation